In [ ]:
%%capture
%run ../ImportsConstantsSparkInit.ipynb
sc.setJobGroup("DA_Precipitation", "DA_Precipitation")

In [ ]:
%%time
pipeline = "[{$match:{'Precipitation-in':{$gte:0, $lte:25.0}}},{ $bucket : { groupBy : '$Precipitation-in', boundaries:[0.0,0.0001,0.1,0.2,0.3,0.4,0.5,1,2,3,7,10,25], output: {count:{$sum:1}}}}, {$sort:{_id:1}}]"
#pipeline = "[{$match:{'Precipitation-in':{$gte:0, $lte:25.0}}},{ $group : { _id:'$Precipitation-in', count:{$sum:1}}}, {$sort:{_id:1}}]"
df = spark.read.format("mongo").option("collection", "TrafficData").option("pipeline", pipeline).load()
df = df.groupby("_id").sum("count")
df = df.withColumnRenamed("_id","Precipitation").withColumnRenamed("sum(count)","AccidentCount")
df = df.orderBy("Precipitation")
#print("Number of Documents: " + str(df.count()))
#df.agg({'AccidentCount': 'sum'}).show()
#df.show(230)

In [ ]:
df.write.format("mongo").option("collection", "AccidentCountByPrecipitation").mode("overwrite").save()

In [ ]:
print("Data Aggregation for Precipitation Data Finished")